# Experiment 0
- The objective with this experiment is to test the pipeline devloped.

## TODOs
1. Use fit_generator() instead of generator()
2. Use H5py format to store features

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from time import time
from tqdm import tqdm
import datetime
import os
import numpy as np
import tensorflow as tf
import keras
import sklearn.model_selection as sk
import copy
from PIL import Image as IM
from keras.callbacks import TensorBoard
import cv2
import json
import pathlib
from glob import glob

Using TensorFlow backend.


In [2]:
## import helpers
from helpers import data_loaders as dls
from helpers import pointcloud as pc
from helpers.viz import plot, plot_history
from helpers.logger import Logger
import utils
## import networks
import lodnn

In [3]:
PATH = '../' # path of the repo.
_NAME = 'experiment0' # name of experiment
!ls $PATH

dataset  nbs  README.md  TODO


In [4]:
# It is better to create a folder with runid in the experiment folder
_EXP, _LOG, _TMP, _RUN_PATH = dls.create_dir_struct(PATH, _NAME)
logger = Logger('EXP0', _LOG + 'experiment0.log')
logger.debug('Logger EXP0 int')
!ls $_EXP

2019_07_12_14_02  2019_07_12_14_54  2019_07_12_17_25  2019_07_12_18_17
2019_07_12_14_16  2019_07_12_17_14  2019_07_12_17_45  log
2019_07_12_14_18  2019_07_12_17_17  2019_07_12_18_03  tmp
2019_07_12_14_19  2019_07_12_17_22  2019_07_12_18_16


In [5]:
# get dataset
train_set, valid_set, test_set = dls.get_dataset(PATH, is_training=True)

In [6]:
KPC = utils.KittiPointCloudClass(train_set=train_set, valid_set=valid_set)

Height ranges from -30.21500015258789 to 2.9130001068115234
Count varies from 0 to 122.0


# Comments
- Use the LoDNN's Ground truth since its better [look at [LoDNN paper](https://arxiv.org/pdf/1703.03613.pdf) Figure 3]
- plot histograms over the normalized features, observe how this changes once we start subsampling
- Use vanilla U-net implementation from [[here](https://github.com/karolzak/keras-unet#Vanilla-U-Net)]

In [7]:
limit_index = -1
# NOTE: change limit_index to -1 to train on the whole dataset
f_train = dls.process_pc(train_set["pc"][0:limit_index], lambda x: KPC.get_features(x))
f_valid = dls.process_pc(valid_set["pc"][0:limit_index], lambda x: KPC.get_features(x))
f_test = dls.process_pc(test_set["pc"][0:limit_index], lambda x: KPC.get_features(x))
gt_train = dls.process_img(train_set["gt_bev"][0:limit_index], func=lambda x: utils.kitti_gt(x))
gt_valid = dls.process_img(valid_set["gt_bev"][0:limit_index], func=lambda x: utils.kitti_gt(x))
gt_test = dls.process_img(train_set["gt_bev"][0:limit_index], func=lambda x: utils.kitti_gt(x))

In [8]:
# _ = plot([[f_train[0][:,:,0], f_train[1][:,:,0],  f_train[2][:,:,0]],
#       [f_train[0][:,:,1], f_train[1][:,:,1],  f_train[2][:,:,1]],
#       [gt_train[0][:,:,0], gt_train[1][:,:,0], gt_train[1][:,:,0]]
#      ])

#### Let the training begin

In [9]:
def get_unique_id():
    return datetime.datetime.today().strftime('%Y_%m_%d_%H_%M')

def create_run_dir(path, run_id):
    path = path.replace("*run_id", str(run_id))
    model_dir = "{}/model/".format(path)
    output_dir = "{}/output/".format(path)
    log_dir = "{}/log/".format(path)
    os.makedirs(model_dir, exist_ok=True)
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(log_dir, exist_ok=True)
    return path

In [10]:
def get_basic_callbacks(path):
    # tensorboard
    # to visualise `tensorboard --logdir="./path_to_log_dir" --port 6006`
    log_path = "{}/log/".format(path)
    tensorboard = TensorBoard(log_dir="{}/{}".format(log_path, time()))
    # save best model
    best_model_path = "{}/model/best_model.h5".format(path) #? .hd5
    save_the_best = keras.callbacks.ModelCheckpoint(filepath=best_model_path,
                                                    verbose=1, save_best_only=True)
    # save models after few epochs
    epoch_save_path = "{}/model/*.h5".format(path)
    save_after_epoch = keras.callbacks.ModelCheckpoint(filepath=epoch_save_path.replace('*', 'e{epoch:02d}-val_acc{val_acc:.2f}'),
                                                       monitor='val_acc', verbose=1, period = 1)
    return [tensorboard, save_the_best, save_after_epoch]

In [11]:
run_id = get_unique_id()
path = create_run_dir(_RUN_PATH, run_id)

In [12]:
model = lodnn.get_model()
# model.summary()
callbacks = get_basic_callbacks(path)

In [13]:
# Add more callbacks
# early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
# reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.000001)
# callbacks = callbacks + [early_stopping, reduce_lr]

In [14]:
# All training params to be added here
training_config = {
    "loss_function" : "binary_crossentropy",
    "learning_rate" : 1e-4,
    "batch_size"    : 1,
    "epochs"        : 10,
    "optimizer"     : "keras.optimizers.Adam"
}

In [15]:
optimizer = eval(training_config["optimizer"])(lr=training_config["learning_rate"])
model.compile(loss=training_config["loss_function"],
              optimizer=optimizer,
              metrics=['accuracy'])

In [19]:
np.array(f_train).T.shape

(6, 200, 400, 228)

In [17]:
# TODO: add fit_generator
m_history = model.fit(x=np.array(f_train),
                      y=np.array(gt_train),
                      batch_size=training_config["batch_size"],
                      epochs=training_config["epochs"],
                      verbose=1,
                      callbacks=callbacks,
                      validation_data=(np.array(f_valid), np.array(gt_valid))) 

Train on 228 samples, validate on 29 samples
Epoch 1/10


UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d_1/convolution}} = Conv2D[T=DT_FLOAT, _class=["loc:@training/Adam/gradients/conv2d_1/convolution_grad/Conv2DBackpropFilter"], data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training/Adam/gradients/conv2d_1/convolution_grad/Conv2DBackpropFilter-0-TransposeNHWCToNCHW-LayoutOptimizer, conv2d_1/kernel/read)]]
	 [[{{node metrics/acc/Mean_1/_261}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_2563_metrics/acc/Mean_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

In [ ]:
model.save("{}/model/final_model.h5".format(path))
plot_history(m_history)

#### Do prediction

In [ ]:
# # predict for one sample
# f_pc_test = dls.process_pc(test_set["pc"][0:1], lambda x: _get_features(x))
# f_pc_test = np.expand_dims(f_pc_test[0], axis=0)
# res_test = model.predict(f_pc_test, verbose=1).squeeze()
# plot([[res_test[:, : , 0]]])

In [17]:
def get_prediction(model, features):
    '''
    Get prediction for a given model and point cloud.
    '''
    f = np.expand_dims(features, axis=0)
    return model.predict(f, verbose=0).squeeze()

In [19]:
points = dls.process_pc(test_set["pc"][0:1], lambda x:x)[0]
features = KPC.get_features(points)
res = get_prediction(model, features)
plot([[res[:, : ,0]]])

UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d_1/convolution}} = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv2d_1/convolution-0-TransposeNHWCToNCHW-LayoutOptimizer, conv2d_1/kernel/read)]]
	 [[{{node activation_1/truediv/_269}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_502_activation_1/truediv", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

In [ ]:
def apply_threshold(res, threshold=0.9):
    '''
    Apply thresholding on probability values.
    '''
    _tmp = res.copy()
    _tmp[res >= threshold] = 1
    _tmp[res < threshold] = 0
    return _tmp.astype(np.uint8)

def apply_argmax(res):
    return np.argmax(res, axis=2)

# Binarization of softmax output from LODNN
- Ideally this threshold needs to be determined by the classifier itself and we perform argmax, for now we can stay with argmax
- An overlay between the prediction and ground truth would be useful to see where the errors are
- Maybe add post processing with morphological filling/reconstruction filters ?

In [ ]:
THRESHOLD = 0.4
plot([[apply_threshold(res, THRESHOLD)[:, : ,0]]])

# Argmax from softmax

In [ ]:
plot([[1-apply_argmax(res)]])

In [ ]:
def get_output(path, model, dataset, threshold, is_viz=False):
    result_path = "{}/output/*".format(path)
    os.makedirs(os.path.dirname(result_path), exist_ok=True)
    # NOTE: Now computing for 8 samples (can this be done on samples from the train and validation set?)
    f = dls.process_pc(dataset["pc"][0:8], lambda x: _get_features(x))
    gt = dls.process_img(dataset["gt_bev"][0:8], func=lambda x: kitti_gt(x))
    F1, P, R, ACC = [], [], [], []
    FN, FP, TP, TN = [], [], [], []
    counter = 0
    diplay_thresh = 3
    display = []
    labels = []
    
    for i, datum in enumerate(zip(f, gt)):
        _f, _gt = datum
        res = get_prediction(model, _f)
        th_res = apply_threshold(res, threshold=threshold)
               
        # get metrics
        p_road = th_res[:, :, 0]
        gt_road = _gt[:, :, 0]
        fn, fp, tp, tn = utils.get_metrics_count(pred=p_road, gt=gt_road)
        f1, recall, precision, acc = utils.get_metrics(gt=gt_road, pred=p_road)
        F1.append(f1)
        P.append(precision)
        R.append(recall)
        ACC.append(acc)
        TP.append(tp)
        FP.append(fp)
        TN.append(tn)
        FN.append(fn)
        
        # for viz
        if is_viz is True:
            display.append([gt_road, res[:,:,0], p_road])
            labels.append(["gt", "Acc: {}".format(str(np.round(acc,2))), "F1: {}".format(str(np.round(f1,2)))])
            if (i+1) % diplay_thresh == 0:
                plot(display, labels, fontsize=10)
                plt.savefig(result_path.replace("*", str(i+1) + ".jpg"))
                display, labels = [], []
    if is_viz is True:
        plot(display, labels, fontsize=10)
        plt.savefig(result_path.replace("*", str(i+1) + ".jpg"))
    
    eps = np.finfo(np.float32).eps        
    _acc = (sum(TP)+sum(TN))/(sum(TP)+sum(FP)+sum(TN)+sum(FN) + eps)
    _recall = sum(TP)/(sum(TP) + sum(FN)+eps)
    _precision = sum(TP)/(sum(TP) + sum(FP)+eps)
    _f1 = 2*((precision * recall)/(precision + recall))    
    return _acc, _recall, _precision, _f1

In [ ]:
_acc, _recall, _precision, _f1 = get_output(path, model, test_set, threshold=THRESHOLD, is_viz=True)
print(_acc, _recall, _precision, _f1)

In [ ]:
details = {"name" : _NAME,
           "run_id" : run_id,
           "dataset": "KITTI",
           "training_config" : training_config,
           "threshold" : THRESHOLD,
           "accuracy" : _acc,
           "recall" : _recall,
           "precision" : _precision,
           "F1" : _f1}

In [ ]:
with open('{}/details.json'.format(path), 'w') as f:
    json.dump(details, f)